> HW7 Example

In [16]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import scipy.special as sc
import os
from IPython.display import display, Image
import plotly.graph_objs as go
from BGBB import *

In [17]:
df = pd.read_csv('../data/BGBB-example.csv')

In [18]:
df.head()

,frequency,recency,num_obs,periods
0,0,0,3179,11
1,1,1,965,11
2,1,2,218,11
3,2,2,503,11
4,1,3,94,11


### Parameter Estimation

In [19]:
alpha, beta, gamma, delta = maximize(df)
alpha, beta, gamma, delta

(1.1368178863457499, 0.8232766418954562, 0.9413082666590868, 5.234122139377152)

### Log-likelihood

> Checking if LL matches with Excel Output

In [20]:
log_likelihood(df, alpha, beta, gamma, delta)

-50715.6623321837

### Probability Alive

In [21]:
p_alive_df = prob_alive_df(df, 11)
p_alive_df.head()

,frequency,recency,p_alive
0,0,0,0.064634
1,1,1,0.020682
2,1,2,0.072415
3,2,2,0.009068
4,1,3,0.154232


### Incremental Tracking Plot

In [22]:
incr_df = pd.read_csv('../data/BGBB-example-1.csv')

In [23]:
incr_df

,Year,Actual Incremental
0,1,5652
1,2,4674
2,3,4019
3,4,3552
4,5,3555
5,6,3163
6,7,3110
7,8,2938
8,9,2703
9,10,2573


In [24]:
expected_df = (
    expected_count(df, 11)
    .assign(
        model=lambda x: x['E[X(n)]'] * 11104,
        annual=lambda x: x['model'].diff()
    )
)
expected_df.loc[0, 'annual'] = expected_df['model'].iloc[0]
expected_df

,n,E[X(n)],model,annual
0,1,0.491576,5458.457784,5458.457784
1,2,0.918664,10200.848218,4742.390434
2,3,1.296578,14397.206077,4196.357859
3,4,1.635722,18163.059189,3765.853112
4,5,1.943493,21580.540928,3417.481739
5,6,2.225339,24710.167641,3129.626713
6,7,2.485396,27597.836295,2887.668655
7,8,2.726873,30279.197733,2681.361438
8,9,2.952315,32782.505481,2503.307748
9,10,3.163773,35130.536957,2348.031475


### Conditional Expectation

In [25]:
ce_df = cond_expectations_df(df, 10)
ce_df.head(10)

,frequency,recency,n,ce
0,0,0,11,0.043772
1,1,1,11,0.026327
2,1,2,11,0.092180
3,2,2,11,0.016946
4,1,3,11,0.196327
5,2,3,11,0.081725
6,3,3,11,0.013430
7,1,4,11,0.332832
8,2,4,11,0.218860
9,3,4,11,0.082292


In [26]:
ce_df.tail()

,frequency,recency,n,ce
62,7,11,11,4.847221
63,8,11,11,5.442936
64,9,11,11,6.038650
65,10,11,11,6.634365
66,11,11,11,7.230079


### c

In [50]:
result = (
    p_alive_df
    .merge(ce_df, on=['frequency', 'recency'])
    .loc[[0, 23, 45, 62, 66],]
    .assign(
        Donor=['Sarah', 'Ayako', 'Sharmila', 'Mary', 'Bob']
    )
)
result = result[['Donor', 'frequency', 'recency', 'p_alive', 'ce']]
result

,Donor,frequency,recency,p_alive,ce
0,Sarah,0,0,0.064634,0.043772
23,Ayako,2,7,0.544006,1.016556
45,Sharmila,9,9,0.132676,0.801185
62,Mary,7,11,1.000000,4.847221
66,Bob,11,11,1.000000,7.230079
